In [ ]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict,Optional
from  langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from  langchain_core.prompts import PromptTemplate

In [40]:
class Q_A(TypedDict):
    question: str
    answer: str
    qna:str
    facts:str
    snapshot:str

In [ ]:
llm = HuggingFaceEndpoint(model="meta-llama/Meta-Llama-3-8B-Instruct",
                          task="text-generation",
                      )
model = ChatHuggingFace(llm=llm)

In [42]:
def give_answer(state:Q_A)->Q_A:
    prompt = PromptTemplate(template="""Answer the question in few words.
Question: {question}
Answer:""",validate_template=True,input_variables=['question'])
    prompt = prompt.invoke({  'question':state['question']})
    result = model.invoke(prompt)
    if type(result.content) == str:
        state['answer'] = result.content
    return state
    


In [43]:
def make_qna(state:Q_A):
 
    prompt = PromptTemplate(template="""Can you make 5 Q and A from this text and make them short : {text}
Answer:""",validate_template=True,input_variables=['text'])
    prompt = prompt.invoke({"text": state["answer"]})

    result = model.invoke(prompt)
    if type(result.content) == str:
        state['qna'] = result.content
    return {'qna':state['qna']}
    


In [44]:
def make_snapshot(state:Q_A):
 
    prompt = PromptTemplate(template="""Can you make 5 Snapshot from this text and make them short : {text}
Answer:""",validate_template=True,input_variables=['text'])
    prompt = prompt.invoke({"text": state["answer"]})

    result = model.invoke(prompt)
    if type(result.content) == str:
        state['snapshot'] = result.content
    return {'snapshot': state['snapshot']}
    


In [ ]:
def write_facts(state:Q_A):
 
    prompt = PromptTemplate(template="""Can you write 5 facts from this text and make them short : {text}
Answer:""",validate_template=True,input_variables=['text'])
    prompt = prompt.invoke({"text": state["answer"]})

    result = model.invoke(prompt)
    if type(result.content) == str:
        state['facts'] = result.content
    return {'facts':state['facts']}
    


In [46]:
graph = StateGraph(Q_A)


In [47]:
def join(state:Q_A)->Q_A:
    return state

In [48]:
graph.add_node('give_answer', give_answer)
graph.add_node('make_qna', make_qna)
graph.add_node('write_facts', write_facts)
graph.add_node('make_snapshot', make_snapshot)
graph.add_node('join', join)

In [49]:
graph.add_edge(START,'give_answer')
graph.add_edge('give_answer','make_qna')
graph.add_edge('give_answer','write_facts')
graph.add_edge('give_answer','make_snapshot')

graph.add_edge('make_qna','join')
graph.add_edge('write_facts','join')
graph.add_edge('make_snapshot','join')

graph.add_edge('join',END)

In [50]:
workflow = graph.compile()


In [51]:
result = workflow.invoke({
    "question": "What is transformers?",
    "answer": "",
    "qna": "",
    'facts':'',
    'snapshot':''})


In [52]:
result

{'question': 'What is transformers?',
 'answer': 'Electrical power transmission.',
 'qna': 'Here are 5 short Q and A on Electrical Power Transmission:\n\n1. **Q:** What is electrical power transmission?\n**A:** The process of sending electrical power from a generating station to a substation or consumer.\n\n2. **Q:** Why is transmission important?\n**A:** To distribute electrical power over long distances efficiently.\n\n3. **Q:** What is the primary medium for transmission?\n**A:** Overhead power lines and underground cables.\n\n4. **Q:** What is the main purpose of power transformers in transmission?\n**A:** To step up voltage for efficient transmission and step down for distribution.\n\n5. **Q:** What is the result of power transmission?\n**A:** Supplying electrical power to consumers and industries.',
 'facts': 'Here are 5 short facts about electrical power transmission:\n\n1. **Long-distance transmission**: Power transmission involves sending electricity over long distances from p